In [6]:
print("ok")

ok


In [ ]:
# -*- coding: utf-8 -*-
"""
简化版：读取 NetCDF 文件并输出 lat/lon/time 基本信息
- 使用 xarray 新写法：decode_times=CFDatetimeCoder(use_cftime=True)
- 更稳健的坐标识别（兼容 lat/lon/latitude/longitude/nav_lat/nav_lon 等）
- 统一字符串化时间，兼容 numpy.datetime64 与 cftime 对象
- 使用 with 上下文自动关闭文件句柄
"""

from pathlib import Path
import xarray as xr

# ========== 用户可改区域 ==========
# 文件目录
DATA_DIR = Path(r"D:\cangku\data")

# 目标文件列表
FILES = [
    "gpp_CanESM5_abrupt-4xCO2_185001-194912_1x1.nc",
    "gpp_Lmon_CanESM5_abrupt-4xCO2_185001-200012.nc",
    "gpp_Lmon_CanESM5_G1_r1i1p2f1_gn_185001-194912.nc",
    "gpp_Lmon_CanESM5_piControl_r1i1p2f1_gn_185001-194912.nc",
    "tos_Omon_CanESM5_abrupt-4xCO2_185001-200012.nc",
    "tos_Omon_CanESM5_G1_r1i1p2f1_gn_185001-194912.nc",
    "tos_Omon_CanESM5_piControl_r1i1p2f1_gn_185001-194912.nc",
]
# =================================

def _format_time_value(v) -> str:
    """将时间值安全地转为字符串，兼容 cftime 与 numpy.datetime64。"""
    try:
        return str(v)
    except Exception:
        try:
            # 某些 cftime 对象可以 .isoformat()
            return v.isoformat()
        except Exception:
            return repr(v)

def _first_last_as_str(da) -> str:
    """将时间坐标的首尾值格式化为 'first → last'。"""
    if da.size == 0:
        return "(empty)"
    first = _format_time_value(da.values[0])
    last = _format_time_value(da.values[-1])
    return f"{first} → {last}"

def _find_coord_name(ds: xr.Dataset, candidates) -> str | None:
    """
    在 ds.coords 中优先匹配候选名；若找不到，再到数据变量中用 attrs 判断。
    返回匹配到的名字或 None。
    """
    # 1) 直接在坐标名里找
    lower_map = {name.lower(): name for name in ds.coords}
    for cand in candidates:
        if cand in lower_map:
            return lower_map[cand]

    # 2) 有些文件把经纬度放到 data_vars，尝试通过 attributes 判断
    for var in list(ds.coords) + list(ds.data_vars):
        v = ds[var]
        std = (v.attrs.get("standard_name", "") or "").lower()
        longn = (v.attrs.get("long_name", "") or "").lower()
        units = (v.attrs.get("units", "") or "").lower()
        # 简单的启发式判断
        if any(k in std for k in ["latitude"]) or any(k in longn for k in ["latitude"]) or units in ("degrees_north", "degree_north"):
            if "lat" in candidates[0]:  # 粗略判断是否在找 lat
                return var
        if any(k in std for k in ["longitude"]) or any(k in longn for k in ["longitude"]) or units in ("degrees_east", "degree_east"):
            if "lon" in candidates[0]:  # 粗略判断是否在找 lon
                return var
    return None

def get_basic_info(path: Path) -> dict:
    """
    打开 NetCDF 并提取基本信息:
      - lat / lon 名称与形状
      - time 名称、长度与范围
    使用新 API：decode_times=CFDatetimeCoder(use_cftime=True) 以支持 360_day / noleap 等日历。
    """
    info = {}
    time_coder = xr.coders.CFDatetimeCoder(use_cftime=True)

    # 使用 with 自动关闭
    with xr.open_dataset(path, decode_times=time_coder) as ds:
        # 统一小写对比
        coord_names_lower = [c.lower() for c in ds.coords]
        # 尝试找到 lat / lon / time
        lat_candidates = ["lat", "latitude", "nav_lat", "y"]
        lon_candidates = ["lon", "longitude", "nav_lon", "x"]
        time_candidates = ["time"]

        # 转成小写构造候选集
        lat_name = _find_coord_name(ds, lat_candidates)
        lon_name = _find_coord_name(ds, lon_candidates)

        # time 相对固定，直接找
        time_name = None
        for c in ds.coords:
            if c.lower() in time_candidates or "time" in c.lower():
                time_name = c
                break

        # 填写 info
        if lat_name is not None and lat_name in ds:
            info["lat_name"] = lat_name
            info["lat_shape"] = tuple(ds[lat_name].shape)
        else:
            info["lat_name"] = None
            info["lat_shape"] = None

        if lon_name is not None and lon_name in ds:
            info["lon_name"] = lon_name
            info["lon_shape"] = tuple(ds[lon_name].shape)
        else:
            info["lon_name"] = None
            info["lon_shape"] = None

        if time_name is not None and time_name in ds:
            t = ds[time_name]
            info["time_name"] = time_name
            info["time_len"] = int(t.size)
            info["time_range"] = _first_last_as_str(t)
        else:
            info["time_name"] = None
            info["time_len"] = 0
            info["time_range"] = "(not found)"

    return info

def main():
    print("=== 基本信息检查开始 ===")
    base = Path(DATA_DIR)
    for fname in FILES:
        path = base / fname
        if not path.exists():
            print(f"[缺失] {fname}")
            continue

        try:
            info = get_basic_info(path)
            print(f"\n文件: {fname}")
            print(f"  lat:  {info.get('lat_name')}    shape={info.get('lat_shape')}")
            print(f"  lon:  {info.get('lon_name')}    shape={info.get('lon_shape')}")
            print(f"  time: {info.get('time_name')}   len={info.get('time_len')}")
            print(f"  time_range: {info.get('time_range')}")
        except Exception as e:
            print(f"[错误] 读取 {fname} 失败：{e}")

    print("\n=== 完成 ===")

if __name__ == "__main__":
    main()


In [1]:
# -*- coding: utf-8 -*-
"""
将给定 NetCDF 文件线性插值到 1°×1° lon-lat 规则网格（lon: 0..359, lat: -90..89）
- 规则网格(1D lat/lon)  → xarray.interp(method="linear")
- 曲线网格(2D lat/lon)  → scipy.interpolate.griddata:
      1) method="linear" 主插值（线性）
      2) 对线性插值 NaN 用 method="nearest" 二次填充（仅填空洞/边界）
- 自动识别坐标名；time 用 CFDatetimeCoder(use_cftime=True)
- 自动排除 vertices/bnds/bounds 顶点与边界变量，仅对物理量插值（优先白名单 tos/gpp）
- 输出到 DATA_DIR/regridded，文件名后缀 _1x1_bilinear.nc
"""

from pathlib import Path
import numpy as np
import xarray as xr

# ========== 用户可改区域 ==========
DATA_DIR = Path(r"D:\cangku\data")
FILES = [
    "gpp_Lmon_CanESM5_abrupt-4xCO2_185001-200012.nc",
    "gpp_Lmon_CanESM5_G1_r1i1p2f1_gn_185001-194912.nc",
    "gpp_Lmon_CanESM5_piControl_r1i1p2f1_gn_185001-194912.nc",
    "tos_Omon_CanESM5_abrupt-4xCO2_185001-200012.nc",
    "tos_Omon_CanESM5_G1_r1i1p2f1_gn_185001-194912.nc",
    "tos_Omon_CanESM5_piControl_r1i1p2f1_gn_185001-194912.nc",
]
OUT_DIR = DATA_DIR / "regridded"
# =================================

# 目标网格
LON_TGT = np.arange(0.0, 360.0, 1.0, dtype=np.float32)   # 0..359
LAT_TGT = np.arange(-90.0, 90.0, 1.0, dtype=np.float32)  # -90..89
GX, GY = np.meshgrid(LON_TGT, LAT_TGT)  # (180, 360)

# ------------------ 工具函数 ------------------
def _normalize_lon_to_0_360(lon):
    return (lon % 360).astype(np.float64)

def _find_coord_name(ds: xr.Dataset, candidates) -> str | None:
    lower_map = {name.lower(): name for name in ds.coords}
    for cand in candidates:
        if cand in lower_map:
            return lower_map[cand]
    # 在 data_vars 中按 attrs 识别
    for var in list(ds.coords) + list(ds.data_vars):
        v = ds[var]
        std = (v.attrs.get("standard_name", "") or "").lower()
        longn = (v.attrs.get("long_name", "") or "").lower()
        units = (v.attrs.get("units", "") or "").lower()
        if ("latitude" in std) or ("latitude" in longn) or (units in ("degrees_north", "degree_north")):
            if any(c.startswith("lat") for c in candidates):
                return var
        if ("longitude" in std) or ("longitude" in longn) or (units in ("degrees_east", "degree_east")):
            if any(c.startswith("lon") for c in candidates):
                return var
    return None

def _is_rectilinear(ds: xr.Dataset, lat_name: str, lon_name: str) -> bool:
    return (ds[lat_name].ndim == 1) and (ds[lon_name].ndim == 1)

def _get_spatial_dims(ds: xr.Dataset, lat_name: str, lon_name: str):
    if _is_rectilinear(ds, lat_name, lon_name):
        return lat_name, lon_name
    lat_dims = ds[lat_name].dims
    lon_dims = ds[lon_name].dims
    if lat_dims != lon_dims or len(lat_dims) != 2:
        raise ValueError(f"曲线网格 lat/lon 维度不匹配: lat={lat_dims}, lon={lon_dims}")
    return lat_dims  # (y_dim, x_dim)

def _select_vars_with_spatial_dims(ds: xr.Dataset, y_dim: str, x_dim: str):
    """
    仅挑选真正需要再网格化的物理量变量：
    - 必须含 y_dim/x_dim
    - 排除名称或维度含 vertices/vertex/bnds/bounds
    - 必须为数值型
    """
    bad_tokens = ("vertices", "vertex", "bnds", "bounds")
    keep = []
    for v in ds.data_vars:
        da = ds[v]
        dims = da.dims
        name_l = v.lower()
        if not ((y_dim in dims) and (x_dim in dims)):
            continue
        if any(tok in name_l for tok in bad_tokens):
            continue
        if any(any(tok in d.lower() for tok in bad_tokens) for d in dims):
            continue
        if not np.issubdtype(da.dtype, np.number):
            continue
        keep.append(v)
    return keep

def _build_encoding_like(ds_vars):
    enc = {}
    for v in ds_vars:
        enc[v] = {"zlib": True, "complevel": 4, "dtype": "f4"}
    return enc

# ------------------ 规则网格：xarray.interp ------------------
def _regrid_rectilinear(ds: xr.Dataset, lat_name: str, lon_name: str) -> xr.Dataset:
    # 经度统一到 [0,360)，并排序
    lon_new = _normalize_lon_to_0_360(ds[lon_name])
    ds = ds.assign_coords({lon_name: lon_new}).sortby(lon_name)

    out = ds.interp({lat_name: LAT_TGT, lon_name: LON_TGT}, method="linear")
    out = out.rename({lat_name: "lat", lon_name: "lon"})
    out["lat"].attrs.update(dict(standard_name="latitude", long_name="Latitude", units="degrees_north"))
    out["lon"].attrs.update(dict(standard_name="longitude", long_name="Longitude", units="degrees_east"))
    return out

# ------------------ 曲线网格：scipy.griddata ------------------
def _regrid_curvilinear_scipy(ds: xr.Dataset, lat_name: str, lon_name: str) -> xr.Dataset:
    from scipy.interpolate import griddata  # 仅在需要时导入

    y_dim, x_dim = _get_spatial_dims(ds, lat_name, lon_name)

    lon2d = _normalize_lon_to_0_360(ds[lon_name].astype(np.float64))
    lat2d = ds[lat_name].astype(np.float64)

    # 输入点坐标 (N, 2)
    P = np.column_stack([lon2d.values.ravel(), lat2d.values.ravel()])

    # 优先白名单（若存在），否则按筛选规则自动选
    preferred = [v for v in ds.data_vars if v.lower() in ("tos", "gpp")]
    var_names = preferred if preferred else _select_vars_with_spatial_dims(ds, y_dim, x_dim)
    if not var_names:
        raise ValueError("未找到需要再网格化的物理量变量。")

    out_vars = {}
    target_shape = (LAT_TGT.size, LON_TGT.size)

    # time 维名（可无）
    time_dim = next((c for c in ds.coords if "time" in c.lower()), None)

    for v in var_names:
        da = ds[v]
        dims = da.dims

        # (time, y, x)
        if (time_dim in dims) and (y_dim in dims) and (x_dim in dims) and (len(dims) == 3):
            tN = ds.sizes[time_dim]
            out_data = np.full((tN, *target_shape), np.nan, dtype=np.float32)
            for ti in range(tN):
                slice2d = da.isel({time_dim: ti}).values.astype(np.float64)
                mask = np.isfinite(slice2d).ravel()
                if mask.sum() >= 3:
                    Zi = griddata(P[mask], slice2d.ravel()[mask], (GX, GY), method="linear")
                    if np.isnan(Zi).any():
                        Znn = griddata(P[mask], slice2d.ravel()[mask], (GX, GY), method="nearest")
                        Zi = np.where(np.isnan(Zi), Znn, Zi)
                    out_data[ti] = Zi.astype(np.float32)

            coords = dict(lon=("lon", LON_TGT), lat=("lat", LAT_TGT))
            if time_dim in ds.coords:
                coords[time_dim] = ds[time_dim]

            out_vars[v] = xr.DataArray(
                out_data, dims=(time_dim, "lat", "lon"),
                coords=coords, attrs=da.attrs,
            )

        # (y, x)
        elif (y_dim in dims) and (x_dim in dims) and (len(dims) == 2):
            slice2d = da.values.astype(np.float64)
            mask = np.isfinite(slice2d).ravel()
            if mask.sum() >= 3:
                Zi = griddata(P[mask], slice2d.ravel()[mask], (GX, GY), method="linear")
                if np.isnan(Zi).any():
                    Znn = griddata(P[mask], slice2d.ravel()[mask], (GX, GY), method="nearest")
                    Zi = np.where(np.isnan(Zi), Znn, Zi)
            else:
                Zi = np.full(target_shape, np.nan, dtype=np.float32)

            out_vars[v] = xr.DataArray(
                Zi.astype(np.float32),
                dims=("lat", "lon"),
                coords=dict(lon=("lon", LON_TGT), lat=("lat", LAT_TGT)),
                attrs=da.attrs,
            )

        else:
            print(f"[跳过] 变量 {v} 维度 {dims} 暂不支持（仅支持 (time,y,x) 或 (y,x)）")

    out = xr.Dataset(out_vars)
    out["lat"].attrs.update(dict(standard_name="latitude", long_name="Latitude", units="degrees_north"))
    out["lon"].attrs.update(dict(standard_name="longitude", long_name="Longitude", units="degrees_east"))

    # 复制 time 等其它坐标
    for c in ds.coords:
        if c not in (lat_name, lon_name, "lat", "lon") and c not in out.coords and c in ds:
            out = out.assign_coords({c: ds[c]})

    return out

# ------------------ 主流程 ------------------
def main():
    OUT_DIR.mkdir(parents=True, exist_ok=True)
    time_coder = xr.coders.CFDatetimeCoder(use_cftime=True)

    for fname in FILES:
        in_path = DATA_DIR / fname
        if not in_path.exists():
            print(f"[缺失] {fname}")
            continue

        try:
            with xr.open_dataset(in_path, decode_times=time_coder) as ds:
                lat_name = _find_coord_name(ds, ["lat", "latitude", "nav_lat", "y"])
                lon_name = _find_coord_name(ds, ["lon", "longitude", "nav_lon", "x"])
                if (lat_name is None) or (lon_name is None):
                    raise ValueError(f"未找到经纬度坐标：lat={lat_name}, lon={lon_name}")

                print(f"\n[处理] {fname} | lat={lat_name} {ds[lat_name].shape}, lon={lon_name} {ds[lon_name].shape}")

                if _is_rectilinear(ds, lat_name, lon_name):
                    out = _regrid_rectilinear(ds, lat_name, lon_name)
                else:
                    out = _regrid_curvilinear_scipy(ds, lat_name, lon_name)

                out_name = in_path.stem + "_1x1_bilinear.nc"
                out_path = OUT_DIR / out_name
                enc = _build_encoding_like(out.data_vars)
                out.to_netcdf(out_path, format="NETCDF4", encoding=enc)
                print(f"[完成] 保存 -> {out_path}")

        except Exception as e:
            print(f"[错误] {fname}: {e}")

    print("\n=== 全部处理结束 ===")

if __name__ == "__main__":
    main()



[处理] gpp_Lmon_CanESM5_abrupt-4xCO2_185001-200012.nc | lat=lat (64,), lon=lon (128,)
[完成] 保存 -> D:\cangku\data\regridded\gpp_Lmon_CanESM5_abrupt-4xCO2_185001-200012_1x1_bilinear.nc

[处理] gpp_Lmon_CanESM5_G1_r1i1p2f1_gn_185001-194912.nc | lat=lat (64,), lon=lon (128,)
[完成] 保存 -> D:\cangku\data\regridded\gpp_Lmon_CanESM5_G1_r1i1p2f1_gn_185001-194912_1x1_bilinear.nc

[处理] gpp_Lmon_CanESM5_piControl_r1i1p2f1_gn_185001-194912.nc | lat=lat (64,), lon=lon (128,)
[完成] 保存 -> D:\cangku\data\regridded\gpp_Lmon_CanESM5_piControl_r1i1p2f1_gn_185001-194912_1x1_bilinear.nc

[处理] tos_Omon_CanESM5_abrupt-4xCO2_185001-200012.nc | lat=latitude (291, 360), lon=longitude (291, 360)


KeyboardInterrupt: 

ok


Saved: D:\cangku\data\GPP.npy (3, 180, 360, 1200) D:\cangku\data\TOS.npy (3, 180, 360, 1200)
